In [329]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from random import randrange
import time
from nltk.stem.porter import PorterStemmer
import spacy
from nltk.stem import WordNetLemmatizer
#nltk.download('all')
import wordnet
nlp = spacy.load('en_core_web_lg',  disable=["parser", "ner"])
import io
import random
import string 
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
from sklearn.metrics.pairwise import cosine_similarity
from nltk import pos_tag
wnl = WordNetLemmatizer()
warnings.filterwarnings('ignore')
from pywsd.utils import lemmatize_sentence

In [9]:
import requests

url = "https://api-football-beta.p.rapidapi.com/players/topscorers"

querystring = {"season":"2022","league":"39"}

headers = {
    "X-RapidAPI-Key": "62a02ca5ccmsha68a5acf6c698d9p1ffc12jsnbe2b9a765139",
    "X-RapidAPI-Host": "api-football-beta.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

#print (type(response.json()))
#print(response.json())
df = response.json()

In [455]:
playerid = 0
goals = df['response'][playerid]['statistics'][0]['goals']['total']
shot = df['response'][playerid]['statistics'][0]['shots']['total']
passing = df['response'][playerid]['statistics'][0]['passes']['total']
playercode = {'Earling Haaland':0, ' Harry Kane':1, 'Aleksandar Mitrovic':2, 'Ivan Toney':3, 'Leandro Trossard':4, ' M. Miguel Almiron':5, 'Phil Foden':6, 'Roberto Firmino':7}

In [460]:
def askplayer(name):
    askq = input("If you want I can tell you about some of the top scorers in the premiere league. So what do you say, Yes or No? ")
    if askq =='yes' or askq == 'Yes':
        print("Here are the names of some of the top scorers I know, Choose any of those names and I'll tell you all about them")
        print("Earling Haaland, Harry Kane, Aleksandar Mitrovic, Ivan Toney, Leandro Trossard, M. Miguel Almiron, Phil Foden, Roberto Firmino")  
       #playerchoose = input("So what's your choice ")
        while True:
            playerchoose = input("So what's your choice ")
            if playerchoose in playercode.keys():
                playerid = (playercode[playerchoose])
                #print(playerid)
                print("hmm...")
                time.sleep(1)
                print("Ah yes... " + playerchoose + " I know all about him. Did you know in this season he has scored a total of " 
                      + str(goals) + " goals and he's attempted a grande total of " + str(shot) +" shots")
                #return False
                break
            else:
                print("That's not one of the names I gave you... Try again.")

In [14]:
 def word_tokenization(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    text_tokenized = word_tokenize(text)
    
    for word in text_tokenized:
        if word in stopwords.words('english'):
            text_tokenized.remove(word)
        
    return(text_tokenized)


In [647]:
def taggandlem(text):
    global lemmatized_list 
    global lmt_string
    entity_list = []
    for word in text:
        if 'NN' in word[1] or 'JJ' in word[1]  or 'VB' in word[1]:
            entity_list.append(word[0])
    
    lemmatized_list = []
    for lit in entity_list:
        lmtz = wnl.lemmatize(lit)
        lemmatized_list.append(lmtz)
        lmt_string = ' '.join(lemmatized_list)

In [480]:
def lem_response_joined(text):
    global lemlist
    number = 0
    lemlist = []
    n=0
    for word in   text:
        number = number+1
     
    while n<number:
        gg =' '.join(text[n])
        lemlist.append(gg)
        n=n+1
    
    #print(lemlist)
    

In [19]:
tokenized_respone = word_tokenization(resp1)
tagged_response = pos_tag(tokenized_respone)

In [568]:
question = [" ",
            "hello, hey, hi, how are you",
            "what is your favourite team",
            "what is your least favourite or most hated team",
            "how long have you been watching football"
                    
                   ]

In [569]:
response = ["Ask another question",
            'Hey, my name is Garry I will be answering your football questions, feel free to ask anything',
                  'My favourite team is Manchester United',
                  'I really hate Manchester City',
                 "I've been watching football since I was two, for as long as I remember I've loved football",    
                 ]

In [570]:
question_list = []
for word in question:
    question_list.append(lemmatize_sentence(word,))
    
lem_response_joined(question_list)
question_lem_list = lemlist

In [571]:
chat_frame = pd.DataFrame()
chat_frame['question'] = question_lem_list
chat_frame['response'] = response

In [648]:
#chat_frame

In [573]:
def response_selection(ui):
    sentence_input = [(ui)]
    similarity_index_list = cosine_similarity(v.fit_transform(chat_frame["question"]), v.transform(sentence_input)).flatten()
    output = chat_frame.loc[similarity_index_list.argmax(), "response"]
    print(output)

In [654]:
def speech(text):
    tag = True
    #exit_command = ['bye','exit,','goodbye']
    while tag == True:
        user_input = input()
        user_input = user_input.lower()
        if user_input == "bye" or  user_input == "exit" or  user_input == "goodbye":
            print("goodbye")
            tag = False
        
        elif "top scorer" in user_input:
            askplayer("name")
            
        else:
            tokenized_respone = word_tokenization(user_input)
            tagged_response = pos_tag(tokenized_respone)
            taggandlem(tagged_response)
            user_input = lmt_string
            #print(user_input)
            response_selection(user_input)

In [655]:
speech("d")

how are you doing
Ask another question
bye
goodbye


In [656]:
tokenized_respone = word_tokenization(input())
tagged_response = pos_tag(tokenized_respone)
taggandlem(tagged_response)
lmt_string

how are you doing


'are doing'

In [ ]:
#redundant

In [256]:
v = TfidfVectorizer()
sentence_input = ["hello"]
similarity_index_list = cosine_similarity(v.fit_transform(data["question"]), v.transform(sentence_input)).flatten()
output = data.loc[similarity_index_list.argmax(), "answer"]

In [46]:
def frequency_of_terms(user_input,responses_option):
    global freqhash
    frequency = 0
    freqhash = []
    n=0
    user_input = user_input.split()
    for word in user_input:
        if word in responses_option:
            frequency +=1
            n=n+1
            
            
            
            
    print(frequency)
    freqhash.append(frequency)

def frequency_of_terms(user_input,responses_option,n):
    global freqhash
    frequency = 0
    freqhash = []
    
 #   user_input = user_input.split()
    for word in user_input:
        if word in responses_option:
            frequency +=1
            
   # if n<3
    frequency_of_terms(user_input,responses_option,(1))
            
            
    print(frequency)
    freqhash.append(frequency)
    
    
    
def frequency_of_terms(user_input,response_option):
    global freqhash
    frequency = 0
    freqhash = []
    n=0
    user_input = user_input.split()
    for word in user_input:
        for item in response_option:
            if response_option[n] <response_option[3]:
                frequency +=1
                if response_option[n] <= response_option[-1]:
                    n=n+1
    print(frequency)
    freqhash.append(frequency)
    
def frequency_of_terms(user_input,responses_option):
    global freqhash
    frequency = 0
    freqhash = []
    n=0
    user_input = user_input.split()
    for word in user_input:
        for item in responses_option:
            if word in item[n]:
                frequency +=1
                n=n+1
    
   # while n<3:
    #    frequency_of_terms(user_input,responses_option)
        
    print(frequency)
    freqhash.append(frequency)

def frequency_of_terms(user_input,responses_option):
    frequency = 0
    user_input = user_input.split()
    for word in user_input:
        for item in responses_option[n] :
            if word in item:
                if responses_option[:]
    print(frequency)

In [ ]:
def lemmatization(text):
    #lemmatized_words = [token.lemma_ for token in nlp(text)]
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    lemmatized_words = [token.lemma_ for token in nlp(text)]

    for word in lemmatized_words:
        if word in stopwords.words('english'):
            lemmatized_words.remove(word)
    
    return text



In [ ]:
#unfinished
def textresponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    tokenized_respone = word_tokenization(user_response)
    tagged_response = pos_tag(tokenized_respone)
    taggandlem(tagged_response)
    

In [ ]:
def taggandlem(text):
    entity_list = []
    for word in text:
        if 'NN' in word[1]:
            entity_list.append(word[0])

    for lit in entity_list:
        print(wnl.lemmatize(lit))
    

In [ ]:
sent_tokens = nltk.sent_tokenize(resp1)
word_tokens = nltk.word_tokenize(resp1)
lmt_string = []


def textresponse(user_response):
    robo_response=''=
    tokenized_response = word_tokenization(user_response)
    tagged_response = pos_tag(tokenized_response)
    taggandlem(tagged_response)
    sent_tokens.append(lmt_string)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    mis_undlist = ["I am sorry! I don't understand you","I didn't quite get that","I'm sorry can you repeat that","I misunderstood that","I'm sorry what"]
    if(req_tfidf==0):
        robo_response=robo_response+mis_undlist[randrange(4)]
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [ ]:
def speech(text):
    userinput = input("Hello my name is Garry, I will be introducing you to the world of football ")
    exit_command = ['bye','exit,','goodbye']
    listintro = ["Do you have anything else that you want to talk about? ","What else do you want to ask?","Is there anything else?","What else"]
    
    for word in exit_command:
        if word in userinput:
            return("You have chosen to end the chat")
    
    
    if "top scorer" in userinput:
        askplayer("example")
        
    
    userinput2 = input(listintro[randrange(4)])
    
    
def speech(text):
    userinput = input("Hello my name is Garry, I will be introducing you to the world of football ")
    exit_command = ['bye','exit,','goodbye']
    listintro = ["Do you have anything else that you want to talk about? ","What else do you want to ask?","Is there anything else?","What else"]
    sent_tokens = nltk.sent_tokenize(resp1)
    word_tokens = nltk.word_tokenize(resp1)
    lmt_string = []
    
    
    
    for word in exit_command:
        if word in userinput:
            return("You have chosen to end the chat")
            exit()
    
    
    if "top scorer" in userinput:
        askplayer("example")
        
    
    userinput2 = input(listintro[randrange(4)])
    
def speech(text):
        tag = True
        exit_command = ['bye','exit,','goodbye']

        user_input = input("type in something")
        for word in user_input.lower().split():
            if word not in exit_command:
                #print("yaaaa")
                sentence_input = [user_input]
                similarity_index_list = cosine_similarity(v.fit_transform(data["question"]), v.transform(sentence_input)).flatten()
                output = data.loc[similarity_index_list.argmax(), "answer"]
                print(output)

        
            else:
                print("goodbye")

In [ ]:
sent_tokens = nltk.sent_tokenize(resp1)
word_tokens = nltk.word_tokenize(resp1)
lmt_string = []
lmt_corp =  nltk.sent_tokenize(lemmatization(resp1))
#may have to reset sent_token every time

def textresponse(user_response):
    robo_response=''
    tokenized_response = word_tokenization(user_response)
    tagged_response = pos_tag(tokenized_response)
    taggandlem(tagged_response)
    sent_tokens.append(lmt_string)
    
    

In [ ]:
def exit_chat(tile):
    exit_command = ['bye','exit,','goodbye']
    for word in exit_command:
        if word in tile:
            return("You have chosen to end the chat")
            return True

In [ ]:
def lem(text):
    global lemmatized_list 
    global lmt_string
    entity_list = []
  #  for word in text:
   #     if 'NN' in word[1]:
    #        entity_list.append(word[0])
    
    lemmatized_list = []
    for lit in text:
        lmtz = wnl.lemmatize(lit)
        lemmatized_list.append(lmtz)
        lmt_string = ' '.join(lemmatized_list)

In [523]:
elist = []
for word in sent_tokens:
    elist.append(lemmatize_sentence(word,))

In [ ]:
sent_tokens = nltk.sent_tokenize(resp1)
word_tokens = nltk.word_tokenize(resp1)
lmt_string = []

In [ ]:
 def word_tokenization(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    text_tokenized = word_tokenize(text)
    
    for word in text_tokenized:
        if word in stopwords.words('english'):
            text_tokenized.remove(word)
        
    return(text_tokenized)


def taggandlem(text):
    global lemmatized_list 
    global lmt_string
    entity_list = []
    for word in text:
        if 'NN' in word[1]:
            entity_list.append(word[0])
    
    lemmatized_list = []
    for lit in entity_list:
        lmtz = wnl.lemmatize(lit)
        lemmatized_list.append(lmtz)
        lmt_string = ' '.join(lemmatized_list)